In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Pré-processamento dos Dados


## Análise Exploratória de Dados

A análise exploratória de dados nos permite caracterizar os dados por meio da análise de medidas estatísticas descritivas extraídas dos dados. 

Esse é um processo importante da modelagem de dados, pois as descobertas feitas a respeito dos dados irão nortear as escolhas sobre os modelos preditivos com os quais iremos trabalhar.

Os objetivos dessa descrição dos dados são:
1. Investigar o comportamento das variáveis
1. Examinar a relação entre variáveis
1. Enfatizar a ordenação de categorias
1. Compreender a estrutura de organização das categorias
1. Explorar a evolução cronológica de uma variável
1. Revelar padrões espaciais nos dados
1. Descrever a conexão entre categorias

## Carregando os dados

In [ ]:
X_full = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv', index_col='Id')
X_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv', index_col='Id')

In [ ]:
X_full.head()

In [ ]:
X_test.head()

## Resumo descritivo

1. **Moda:** O valor mais frequente. Normalmente utilizada para variáveis nominais.
1. **Média aritmética simples:** soma dos valores divida pelo número de elementos.
1. **Mediana:** é o valor central do conjunto de valores quando ordenados de forma crescente. Em caso de número par, será a média dos dois valores centrais da amostra.
1. **Variância:** é a distância quadrática média em relação à média. E[(X-E(X))²]
1. **Desvio-padrão:** é uma medida de dispersão em torno da média da variável. Medida pela raiz quadrada da variância.

In [ ]:
# X_full.info()

In [ ]:
#X_test.info()

In [ ]:
X_full.describe()

In [ ]:
X_test.describe()

In [ ]:
X_full.describe(exclude=np.number)

In [ ]:
X_test.describe(exclude=np.number)

## Análise de Correlação

Permite verificar o grau de relação entre variáveis.

**Pearson:** indica o grau de relação linear entre variáveis quantitativas simétricas. >0 indica associação positiva, ou seja as duas variáveis crescem juntas; <0 indica associação negativa, na medida que uma cresca a outra decresce. ~0 indica ausência de relação linear.

**Spearman:** avalia relações monotônicas lineares ou não entre variáveis contínuas ou ordinais. É calculada aplicando a medida do coeficiente de Pearson aplicado aos valores da ordem de duas variáveis. Não é sensível a assimetrias na distribuição.

Rodrigues, Francisco. Análise Exploratória de Dados: Correlação de Pearson e Spearman. Disponível em: https://youtu.be/qqRUsY2Fu0A

https://towardsdatascience.com/clearly-explained-pearson-v-s-spearman-correlation-coefficient-ada2f473b8

1. Histograma: é uma representação da função de distribuição de probabilidades. (b.h=probabilidade|f(x).x=p(x))
1. Gráfico de frequência absoluta ou relativa: indica a distribuição das variáveis.
1. Relação entre variávies

https://seaborn.pydata.org/generated/seaborn.distplot.html

https://seaborn.pydata.org/tutorial/distributions.html

https://seaborn.pydata.org/tutorial/relational.html


In [ ]:
import seaborn as sns

In [ ]:
X_full.corr(method='spearman').SalePrice.sort_values(ascending=False)

In [ ]:
X_full.OverallQual.hist()

In [ ]:
sns.pairplot(X_full[['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 
                    'YearBuilt', 'GarageArea', 'FullBath','TotalBsmtSF']])

In [ ]:
sns.pairplot(X_test[['OverallQual', 'GrLivArea', 'GarageCars', 
                    'YearBuilt', 'GarageArea', 'FullBath','TotalBsmtSF']])

## Análise de Outliers (Medidas de posição relativa)

Quartis e percentis são medidas relativas pois são calculadas em relação à mediana da amostra. 
P25 = Q1; P50 = Q2; PN=maior que N% dos valores observados.

**Box plots:** são gráficos que apresentam de forma visual as medidas de posição relativa.

|![Different parts of a boxplot - Towards Data Science](https://miro.medium.com/max/18000/1*2c21SkzJMf3frPXPAR_gZA.png)|
|:--:| 
| *Different parts of a boxplot - [Towards Data Science](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51)*|

In [ ]:
sns.boxplot(x=X_full.SalePrice)

In [ ]:
X_full.SalePrice.describe()

In [ ]:
SalePrice_outlier_conditional = X_full.SalePrice < 450000

X_full_without_outliers = X_full[SalePrice_outlier_conditional ]

In [ ]:
sns.boxplot(x=X_full_without_outliers.SalePrice)

In [ ]:
sns.boxplot(x=X_full_without_outliers.OverallQual)

Queremos eliminar as observações que estejam fora dos limites máximo e mínimo do boxplot, ou seja se o valor é menor que (Q1 - 1.5IQR) ou maior que (Q3 + 1.5IQR).

In [ ]:
OverallQual_Q1 = X_full.OverallQual.quantile(0.25)
OverallQual_Q3 = X_full.OverallQual.quantile(0.75)
OverallQual_IQR = OverallQual_Q3-OverallQual_Q1
OverallQual_outlier_conditional =  ((X_full.OverallQual > OverallQual_Q1 - 1.5*OverallQual_IQR) & (X_full.OverallQual < OverallQual_Q3 + 1.5*OverallQual_IQR))

In [ ]:
X_full_without_outliers = X_full_without_outliers[OverallQual_outlier_conditional]

In [ ]:
sns.boxplot(x=X_full_without_outliers.OverallQual)

In [ ]:
sns.boxplot(x=X_full.GrLivArea)

In [ ]:
sns.boxplot(x=X_full_without_outliers.GrLivArea)

In [ ]:
sns.boxplot(x=X_full.GarageCars)

In [ ]:
sns.boxplot(x=X_full_without_outliers.GarageCars)

In [ ]:
GarageCars_Q1 = X_full.GarageCars.quantile(0.25)
GarageCars_Q3 = X_full.GarageCars.quantile(0.75)
GarageCars_IQR = GarageCars_Q3-GarageCars_Q1
GarageCars_outlier_conditional =  ((X_full.GarageCars > GarageCars_Q1 - 1.5*GarageCars_IQR) & (X_full.GarageCars < GarageCars_Q3 + 1.5*GarageCars_IQR))

In [ ]:
X_full_without_outliers = X_full_without_outliers[GarageCars_outlier_conditional]

In [ ]:
sns.boxplot(x=X_full_without_outliers.GarageCars)

In [ ]:
sns.boxplot(x=X_full_without_outliers.YearBuilt)

In [ ]:
sns.boxplot(x=X_full_without_outliers.GarageArea)

In [ ]:
sns.boxplot(x=X_full_without_outliers.FullBath)

In [ ]:
sns.boxplot(x=X_full_without_outliers.TotalBsmtSF)

In [ ]:
sns.histplot(X_full_without_outliers.SalePrice)

## Substituição de valores ausentes

https://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values

In [ ]:
ordinal_features = ['ExterCond', 'BsmtQual','BsmtCond', 'HeatingQC', 'KitchenQual',
                    'FireplaceQu','GarageQual', 'GarageCond','PoolQC']
nominal_features= list(set(X_full_without_outliers.select_dtypes(include='object').columns.values) - set(ordinal_features))

In [ ]:
X = X_full_without_outliers.drop(['SalePrice'], axis=1)
feature_names = X_full_without_outliers.columns.values
y = X_full_without_outliers.SalePrice

X_nums = X.select_dtypes(exclude='object')
X_cats = X.select_dtypes(include='object')
X_cats_names = X_cats.columns.values

X_test_nums = X_test.select_dtypes(exclude='object')
X_test_cats = X_test.select_dtypes(include='object')
X_test_cats_names = X_test_cats.columns.values

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp_mean = SimpleImputer(strategy='median')
imp_most = SimpleImputer(strategy='most_frequent')

In [ ]:
X_cats = imp_most.fit_transform(X_cats)
X_nums = imp_mean.fit_transform(X_nums)
X_test_cats = imp_most.transform(X_test_cats)
X_test_nums = imp_mean.transform(X_test_nums)

## Tratando variáveis categóricas

https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features

https://contrib.scikit-learn.org/category_encoders/

1. Exclusão ou Substituição
1. One-hot Encoding
1. Ordinal Encoding
1. Label Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [ ]:
one_hot_enc = OneHotEncoder(handle_unknown='ignore')
ordinal_enc = OrdinalEncoder(handle_unknown='ignore')

In [ ]:
X_cats_nominal = one_hot_enc.fit_transform(pd.DataFrame(X_cats, columns=[X_cats_names])[nominal_features])
X_test_cats_nominal = one_hot_enc.transform(pd.DataFrame(X_test_cats, columns=[X_cats_names])[nominal_features])

In [ ]:
X_cats_ordinal = ordinal_enc.fit_transform(pd.DataFrame(X_cats, columns=[X_cats_names])[ordinal_features])
X_test_cats_ordinal = ordinal_enc.transform(pd.DataFrame(X_test_cats, columns=[X_cats_names])[ordinal_features])

In [ ]:
X_cats_nominal.shape

In [ ]:
X_cats_ordinal.shape

In [ ]:
X_nums.shape

In [ ]:
from scipy.sparse import hstack

In [ ]:
X_transformed = hstack((X_cats_nominal,X_cats_ordinal, X_nums))
X_test_transformed = hstack((X_test_cats_nominal,X_test_cats_ordinal, X_test_nums))

**Nota:** existem módulos como sweetviz ou pandas-profiling que podem auxiliar nessa análise.

## Normalização dos dados
https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
max_scaler = MaxAbsScaler()

In [ ]:
X_transformed_max = max_scaler.fit_transform(X_transformed)
X_test_transformed_max = max_scaler.transform(X_test_transformed)

## Padronização dos dados

https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling

Características de uma distribuição normal
* Mádia: 0
* Variância: 1

In [ ]:
from sklearn import preprocessing

In [ ]:
std_scaler = preprocessing.StandardScaler(with_mean = False)

In [ ]:
std_scaler.fit(X_transformed_max)
X_transformed_stand = std_scaler.transform(X_transformed_max)
X_test_transformed_stand = std_scaler.transform(X_test_transformed_max)

In [ ]:
X_transformed_stand.shape

Pularemos a etapa de seleção do modelo nessa aula.

## Separando dados de treino e validação

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_transformed_stand, y, train_size=0.8, random_state=3)

## Treinando modelo

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
house_prices_model = DecisionTreeRegressor(random_state=0,
                                           criterion='mse')

In [ ]:
house_prices_model.fit(X_train,y_train)

## Validando o modelo

In [ ]:
val_predictions = house_prices_model.predict(X_val)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
mean_squared_error(y_val, val_predictions, squared=False)

In [ ]:
mean_absolute_error(y_val, val_predictions)

In [ ]:
pd.DataFrame({'y_val': y_val,
            'val_pred': val_predictions,
             'abs_err': abs(val_predictions-y_val)}).describe()

Baseline: 

1. RMSE- 41206.010178103206
1. MAE- 24401.085616438355

## Importância de variáveis

https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e

In [ ]:
house_prices_model.feature_importances_

In [ ]:
# feature_importance = pd.DataFrame({'feature_name': feature_names,
#                       'feature_importance': house_prices_model.feature_importances_})


In [ ]:
# feature_importance.sort_values(by='feature_importance', ascending=False)

In [ ]:
from sklearn import tree
import graphviz

tree_graph = tree.export_graphviz(house_prices_model, out_file=None)
graphviz.Source(tree_graph)

## Generalizando para os dados de teste

In [ ]:
test_predictions = house_prices_model.predict(X_test_transformed_stand)

In [ ]:
test_predictions

## Criando arquivo de submissão

In [ ]:
output = pd.DataFrame({'Id': X_test.index,
                      'SalePrice': test_predictions})
output.set_index('Id', inplace=True)
output.to_csv('submission.csv')

In [ ]:
output.describe()

O arquivo exportado será submetido à plataforma.